### Initialization

Import of standard and third party libraries.

In [1]:
from pathlib import Path
import sys

import astroid
import owlready2 as owl

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


Adding `codeontology` to sys path for its import.

In [2]:
codeontology_path = Path("../../../../").resolve()
assert codeontology_path.exists()
sys.path.insert(0, str(codeontology_path))

Loading the ontology.

In [3]:
ontology_path = Path("../../../ontology/codeontology.owl").resolve()
assert ontology_path.exists()
ontology = owl.get_ontology(str(ontology_path)).load()
ontology.base_iri = r"http://rdf.webofcode.org/woc/"

### Code sample

In [4]:
node = astroid.extract_node("""
def g(
    pos_only_1: 'list[int]' = 1,
    /,
    normal_1: 'ann_normal_1' = 'def_normal_1',
    *var_args: 'ann_var_args',
    kw_only_1: 'ann_kw_only_1' = 'def_kw_only_1',
    **kw_args: 'ann_kw_args'
):
    pass
""")
args_node = node.args

### Planning the transformation

In [16]:
assert type(args_node) is astroid.Arguments
from codeontology.rdfization.python3.extract.transformer.tracking import resolve_annotation

parent_function_node = args_node.parent
assert type(parent_function_node) in [astroid.FunctionDef, astroid.AsyncFunctionDef]

# Parameters field in the `args_node` ordered by position in the signature
args_fields = [
    # arg_field_name, ann_field_name,            is_positional, is_list, is_var_arg
    ("posonlyargs",   "posonlyargs_annotations", True,          True,    False,),
    ("args",          "annotations",             True,          True,    False,),
    ("vararg",        "varargannotation",        True,          False,   True,),
    ("kwonlyargs",    "kwonlyargs_annotations",  False,         True,    False,),
    ("kwarg",         "kwargannotation",         False,         False,   True,),
]

# Read all the parameters and accumulate them
parameters = []
pos = 0
is_var_args = False

print(args_node.defaults)
print(args_node.kw_defaults)
print()

for arg_field_name, ann_field_name, is_positional, is_list, is_var_arg in args_fields:
    arg_field = getattr(args_node, arg_field_name)
    ann_field = getattr(args_node, ann_field_name)
    if not is_list:
        arg_field = [arg_field] if arg_field is not None else []
        ann_field = [ann_field] if ann_field is not None else []
        
    print(arg_field)
    print(ann_field)
    print()

[<Const.str l.5 at 0x21469e62f40>]
[<Const.str l.7 at 0x21469e62e50>]

[<AssignName.pos_only_1 l.3 at 0x21469e62af0>]
[<Const.str l.3 at 0x21469e62c70>]

[<AssignName.normal_1 l.5 at 0x21469e62c10>]
[<Const.str l.5 at 0x21469e62eb0>]

['var_args']
[<Const.str l.6 at 0x21469e62bb0>]

[<AssignName.kw_only_1 l.7 at 0x21469e62df0>]
[<Const.str l.7 at 0x21469e628b0>]

['kw_args']
[<Const.str l.8 at 0x21469e62cd0>]



In [46]:
astroid.extract_node("a")

<Name.a l.1 at 0x21469ed0100>

In [5]:
from codeontology.rdfization.python3.extract.transformer import Transformer

In [6]:
Transformer.visit_to_transform(node)

In [7]:
node.is_var_args

True

In [8]:
for name, pos, type_, description, is_vararg, is_pos_only, is_key_only in args_node.params:
    print(name)
    print(pos)
    print(type_)

pos_only_1
0
(<ClassDef.list l.0 at 0x20417816580>, <ClassDef.int l.0 at 0x204177cfb20>)
normal_1
1
ClassDef.str(name='str',
             doc=("str(object='') -> str\n"
              'str(bytes_or_buffer[, encoding[, errors]]) -> str\n'
              '\n'
              'Create a new string object from the given object. If '
              'encoding or\n'
              'errors is specified, then the object must expose a data '
              'buffer\n'
              'that will be decoded using the given encoding and error '
              'handler.\n'
              'Otherwise, returns the result of object.__str__() (if '
              'defined)\n'
              'or repr(object).\n'
              'encoding defaults to sys.getdefaultencoding().\n'
              "errors defaults to 'strict'."),
             is_dataclass=False,
             position=None,
             decorators=None,
             bases=[<Name.object l.0 at 0x20417935940>],
             keywords=[],
             doc_node=<Cons

In [9]:
node = astroid.extract_node("""
def g(a: List[str | int]):
    pass
""")
args_node = node.args

In [10]:
arg = node.args.annotations[0]

In [11]:
arg

<Subscript l.2 at 0x204151f9220>

In [12]:
astroid.extract_node("List[str | int]")

<Subscript l.1 at 0x204151f9730>

In [13]:
node.args.parent

<FunctionDef.g l.2 at 0x204151f9430>

In [14]:
node = astroid.extract_node("'a string' + 'b'")

In [15]:
dir(list(node.get_children())[0])

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_astroid_fields',
 '_explicit_inference',
 '_fixed_source_line',
 '_get_assign_nodes',
 '_get_name_nodes',
 '_get_return_nodes_skip_functions',
 '_get_yield_nodes_skip_lambdas',
 '_infer',
 '_infer_name',
 '_other_fields',
 '_other_other_fields',
 '_proxied',
 '_repr_name',
 '_wrap_attr',
 'accept',
 'as_string',
 'block_range',
 'bool_value',
 'callable',
 'child_sequence',
 'col_offset',
 'display_type',
 'end_col_offset',
 'end_lineno',
 'eq',
 'frame',
 'fromlineno',
 'get_children',
 'getattr',
 'getitem',
 'has_base',
 'has_dynamic_getattr',
 'igetattr',
 'infer',
 'infer_binary_

In [16]:
list(node.get_children())[0].pytype()

'builtins.str'